# Part 1 : Natural Language Processing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split

%matplotlib inline

## 1.1 Data

* In this lecture, we use the SMS Spam Collection Data Set from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection). 
    * A collection of 425 SMS spam messages was manually extracted from the Grumbletext Web site.
    * A subset of 3,375 SMS randomly chosen ham messages of the NUS SMS Corpus (NSC), which is a dataset of about 10,000 legitimate messages collected for research at the Department of Computer Science at the National University of Singapore.

In [ ]:
#df_sms = pd.read_csv('./SMS_Spam.tsv', sep='\t')

import io
from google.colab import files
uploaded = files.upload()

df_sms = pd.read_csv(io.StringIO(uploaded['SMS_Spam.tsv'].decode('utf-8')), sep='\t')

In [ ]:
df_sms.head()

## 1.2 Exploratory Data Analysis
* First, how many messages the data have?

In [ ]:
len(df_sms)

* Then, now, how many spams and hams each other?

In [ ]:
df_sms['label'].value_counts()

* Now, let's apply lengths of each message and create a new column.

In [ ]:
df_sms['length'] = df_sms['message'].apply(len)

In [ ]:
df_sms.head()

* How are the lengths of messages distributed?

In [ ]:
sns.displot(df_sms['length'])
plt.show()

* Are there any differences of the distribution of spam and ham messages?

In [ ]:
df_spam = df_sms[df_sms['label']=='spam'].reset_index(drop=True)
df_ham = df_sms[df_sms['label']=='ham'].reset_index(drop=True)

In [ ]:
plt.figure(figsize=(15,10))

sns.displot(df_spam['length'], color='red')
sns.displot(df_ham['length'], color='blue')
plt.legend(labels=['spam','ham'])
plt.show()

## 1.3 Text preprocessing
* For analyzing texts, we need to split each message into individual words.
* Let's remove punctuations first.
    * Python's built-in library **string** would provide a quick and convenient way of removing them.

In [ ]:
import string

string.punctuation

* Check characters whether they are punctuations or not.

In [ ]:
sample = "Hello! This is Hyundai Card / Capital / Commercial - KAIST Ditigal Finance course"

In [ ]:
sample_nopunc = []
for char in sample:
    if char not in string.punctuation:
        sample_nopunc.append(char)

In [ ]:
sample_nopunc = "".join(sample_nopunc)

* Now, it's a step to remove stopwords. The NLTK library is a kind of stardard library for processing texts in Python (https://www.nltk.org/).
* The NLTK library provide a list of stopwords.

In [ ]:
import nltk
from nltk.corpus import stopwords

* We can specify a language for stopwords list.

In [ ]:
nltk.download('stopwords')

In [ ]:
stopwords.words('english')

* Split the message and remove stopwords according to the list.

In [ ]:
sample_nopunc

In [ ]:
sample_nopunc.split()

In [ ]:
remove_stopwords = []
for word in sample_nopunc.split():
    if word.lower() not in stopwords.words('english'):
        remove_stopwords.append(word)

In [ ]:
remove_stopwords

* When you make a function for this, it would be more useful to apply it later.

In [ ]:
def preprocessing(text):
    
    # remove punctuation
    nopunc = []
    for char in text:
        if char not in string.punctuation:
            nopunc.append(char)
            
    nopunc = "".join(nopunc)
    
    # remove stopwords
    remove_stop = []
    for word in nopunc.split():
        if word.lower() not in stopwords.words('english'):
            remove_stop.append(word)
            
    # remove words less than three characters
    tokens = []
    for word in remove_stop:
        if len(word) >= 3:
            tokens.append(word)
            
    #tokens = " ".join(tokens)
    
    return tokens

In [ ]:
sample

In [ ]:
preprocessing(sample)

* You can apply the preprocessing function to whole dataframe.

In [ ]:
df_sms.head()

In [ ]:
df_sms['message'].apply(preprocessing)

## 1.4 Frequency Analysis

In [ ]:
clean_spam = df_spam['message'].apply(preprocessing)
clean_ham = df_ham['message'].apply(preprocessing)

* First, let's merge whole values of each dataframe into one list.

In [ ]:
whole_spam = []
for line in clean_spam.tolist():
    whole_spam += line
    
whole_ham = []
for line in clean_ham.tolist():
    whole_ham += line

* The **Text** class in **NLTK** library provide some useful methods to text analysis.

In [ ]:
from nltk import Text

ham_text = Text(whole_ham)
spam_text = Text(whole_spam)

* The **vocab** method in the **Text** class can extract the frequency of usage for each token.

In [ ]:
freqDist_ham = ham_text.vocab()

In [ ]:
freqDist_ham.most_common(10)

* How about spam messages?

In [ ]:
freqDist_spam = spam_text.vocab()
freqDist_spam.most_common(10)

* You can plot the distribution of each token by the **plot** method.

In [ ]:
plt.figure(figsize=(10,8))

ham_text.plot(30)
plt.show()

* We can also use the **wordcloud** package for visualization. 
* You can download the package by `conda install -c conda-forge wordcloud`

In [ ]:
from wordcloud import WordCloud

plt.figure(figsize=(15,10))

wc_ham = WordCloud(width=1000, height=600, background_color="black", random_state=0)
plt.imshow(wc_ham.generate_from_frequencies(freqDist_ham))
plt.axis("off")
plt.show()

# Part 2 : Recommendation System

* Recommendation system is a sort of information filtering system that seeks to predict the "rating" or "preference" a user would give to an item. They are primarily used in commercial applications (https://en.wikipedia.org/wiki/Recommender_system)
* There are two common types of recommender systems:
    * **Content-Based Filtering** focus on the attributes of the items and give you recommendations based on the similarity between them.
    
    * **Collaborative Filtering** produces recommendations based on the user's attitude (activity) to items.


* Movie recommendation is one of the first step to start learning recommendation systems.
* MovieLens dataset is a famous one for learning to build the recommendation systems.
    * https://grouplens.org/datasets/movielens/
    * https://kaggle.com/grouplens/movielens-20m-dataset

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#movies = pd.read_csv('./movies.csv')

uploaded = files.upload()
movies = pd.read_csv(io.StringIO(uploaded['movies.csv'].decode('utf-8')))

* Let's produce a content-based filtering based on genre similarity.

In [ ]:
vectorizer = CountVectorizer()
genre_vec = vectorizer.fit_transform(movies['genres'])

In [ ]:
sim_mat = cosine_similarity(genre_vec, genre_vec)

In [ ]:
genre_sim = pd.DataFrame(index=movies['title'], columns=movies['title'], data=sim_mat)

* Now, let's import one more dataset "ratings.csv", and produce collaborative filtering.

In [ ]:
#ratings = pd.read_csv('./ratings.csv')

uploaded = files.upload()
ratings = pd.read_csv(io.StringIO(uploaded['ratings.csv'].decode('utf-8')))

In [ ]:
ratings.head()

* Let's merge those two dataframes.

* Which movie has the highest user ratings on average?

* Which movies received the most ratings from users?

* Let's combine of those two results.

In [ ]:
movie_ratings = pd.DataFrame(df_movies.groupby('title')['rating'].mean())
movie_ratings['numbers'] = pd.DataFrame(df_movies.groupby('title')['rating'].count())
movie_ratings.head()

* Now, reshape the dataframe with using pivot_table.

* Fill the NaN values to 0.

* Let's take two examples of movies.

In [ ]:
Matrix = user_movie_matrix['Matrix, The (1999)']
Matrix.head(10)

In [ ]:
Terminator = user_movie_matrix['Terminator 2: Judgment Day (1991)']
Terminator.head(10)

* How similar with those two movies?

* Which movie is the most similar with the "Matrix, The (1999)"?

In [ ]:
Matrix_corr = pd.DataFrame(user_movie_matrix.corrwith(Matrix), columns=['correl'])

In [ ]:
Matrix_corr.sort_values(by='correl', ascending=False).head(10)

* Now, let's use one more metric for similarity - Cosine similarity

In [ ]:
movie_user_matrix = df_movies.pivot_table(index='title', columns='userId', values='rating')
movie_user_matrix.head()

In [ ]:
movie_user_matrix.fillna(0, inplace=True)

item_based_filter = cosine_similarity(movie_user_matrix)

In [ ]:
item_based_matrix = pd.DataFrame(index=movie_user_matrix.index, columns=movie_user_matrix.index, data=item_based_filter)

# Part 3 : Cluster analysis

* Clustering is a Machine Learning technique that involves the grouping of data points.
    * https://en.wikipedia.org/wiki/Cluster_analysis
    * https://developers.google.com/machine-learning/clustering/clustering-algorithms
* K-means clustering is one of the simplest and popular unsupervised machine learning algorithms that will attempt to group similar clusters together in data (https://en.wikipedia.org/wiki/K-means_clustering).
* You can also check out further materials to learn.
    * http://www.mit.edu/~9.54/fall14/slides/Class13.pdf
    * https://www.youtube.com/watch?v=Ev8YbxPu_bQ
    * https://www.youtube.com/watch?v=hDmNF9JG3lo
    * https://www.coursera.org/learn/data-science-k-means-clustering-python#syllabus

* Let's apply the k-means algorithm to the sample data.

In [ ]:
#data = pd.read_csv('./samples.csv')

uploaded = files.upload()
data = pd.read_csv(io.StringIO(uploaded['samples.csv'].decode('utf-8')))

In [ ]:
sns.relplot(x='V1', y='V2', data=data, kind='scatter')
plt.show()

* Of course, the scikit-learn package provide model for k-means algorithm.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans_fit = kmeans.fit(data)

* How about applying a different K?

* How can we decide the best $K$ value? 

In [ ]:
sse = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k).fit(data)
    sse[k] = kmeans.inertia_

In [ ]:
x = list(sse.keys())
y = list(sse.values())

plt.plot(x, y)
plt.show()

* Now, let's take a look at the customer segmentation data,
    * https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python

In [ ]:
#customers = pd.read_csv('./Mall_Customers.csv')

uploaded = files.upload()
customers = pd.read_csv(io.StringIO(uploaded['Mall_Customers.csv'].decode('utf-8')))

* Let's apply clustering algorithm with two features, annual income and spending score.

In [ ]:
sns.scatterplot(x = 'Annual_Income', y = 'Spending_Score',  data = customers)
plt.show()

In [ ]:
data = customers[['Annual_Income', 'Spending_Score']]

In [ ]:
kmeans = KMeans(n_clusters=5)
kmeans_fit = kmeans.fit(data)
labels = kmeans_fit.predict(data)

In [ ]:
customers['cluster'] = labels

In [ ]:
sns.relplot(x='Annual_Income', y='Spending_Score', hue='cluster',data=customers)
plt.show()

In [ ]:
sse = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k).fit(data)
    sse[k] = kmeans.inertia_

In [ ]:
x = list(sse.keys())
y = list(sse.values())

plt.plot(x, y)
plt.show()

* One more thing with saving the trained model and reload to predict.
    * You can produce a kind of Web App with streamlit (https://streamlit.io/)

In [ ]:
from joblib import dump, load

dump(kmeans, './Mall_Customers_clusters.joblib')

In [ ]:
# load the model
model = load('./Mall_Customers_clusters.joblib')

In [ ]:
income = 80
score = 20

row = [income, score]
feat_cols = data.columns

df = pd.DataFrame([row], columns = feat_cols)
features = pd.DataFrame(df, columns = feat_cols)

In [ ]:
model.predict(features)